In [3]:
import time
import numpy as np
import pandas as pd
from utils import mysqlDatabase, dcardApi

In [4]:
class commentCrawler:
    def __init__(self, database_username, database_password, database_ip, database_name, base_url, popular, max_limit):
        self.database_username = database_username
        self.database_password = database_password
        self.database_ip = database_ip
        self.database_name = database_name
        self.base_url = base_url
        self.popular = popular
        self.max_limit = max_limit
    def get_postid_list(self, MysqlDatabase, sql):
        df = MysqlDatabase.select_table(sql)
        postid_list = df['postid'].tolist()
        return postid_list
    def get_comment(self, DcardApi, postid):
        df = DcardApi.get_Dcard_posts_comments(postid)
        return df
    def main(self):
        '''
        實作
        '''
        MysqlDatabase = mysqlDatabase(self.database_username, self.database_password, self.database_ip, self.database_name)
        DcardApi = dcardApi(self.base_url, self.popular, self.max_limit)
        ## 文章列表
        sql = '''SELECT distinct dp.postid, dp.title 
        FROM Bigdata.dcard_posts dp
        where dp.commentCount >= 30
        and not exists(
            select id from Bigdata.dcard_comments dc where dc.id = dp.postid
        )
        '''
        postid_list = self.get_postid_list(MysqlDatabase, sql)
        print('length of postid_list: ', len(postid_list))
        ## 打api拿留言
        df_comment = pd.DataFrame(columns=['id', 'anonymous', 'postId', 'createdAt', 'floor', 'content', 'likeCount', 'withNickname', 'gender', 'school', 'department'])
        time_start = time.time() 
        for postid in postid_list:
            time.sleep(1)
            time_cost = time.time() - time_start
            if time_cost >= 3600:
                break
            try:
                print(postid)
                df_comment = pd.concat([df_comment, self.get_comment(DcardApi, postid)])
            except:
                print('connection abort, need some break')
                time.sleep(5)
                continue
        print('total time cost(sec): ', time_cost)
        df_comment = df_comment.fillna('')[['id', 'anonymous', 'postId', 'createdAt', 'floor', 'content', 'likeCount', 'withNickname', 'gender', 'school', 'department']]
        MysqlDatabase.upsert_table(df_comment, table_name='dcard_comments')
        return df_comment

In [5]:
if __name__ == '__main__':
    database_username = 'jackyfu'
    database_password = 'data8756'
    database_ip       = 'sg2nlmysql29plsk.secureserver.net'
    database_name     = 'Bigdata'
    base_url = 'https://www.dcard.tw/service/api/v2'
    popular = 'false'
    max_limit = '100'
    ###
    CommentCrawler = commentCrawler(database_username, database_password, database_ip, database_name, base_url, popular, max_limit)
    df_comment = CommentCrawler.main()

Successfully select from Bigdata table
length of postid_list:  364
236808962
236807724
236682003
236806608
236805307
236803687
connection abort, need some break
236806003
236808552
236809513
236810114
236806873
236808339
236805438
236811796
236806384
236807624
236809533
236804662
236804236
236806522
236809074
connection abort, need some break
236806805
236807847
236804628
236810018
236812367
236806184
236804584
236809648
236810251
236807801
connection abort, need some break
236809544
connection abort, need some break
236811779
connection abort, need some break
236768502
connection abort, need some break
236804373
connection abort, need some break
236806177
connection abort, need some break
236806398
connection abort, need some break
236807669
connection abort, need some break
236810998
connection abort, need some break
236813642
connection abort, need some break
236808999
connection abort, need some break
236805080
connection abort, need some break
236805821
connection abort, need some